# Solving Sudokus with Binaries

__author__ = "Rahul Kakodkar"
__copyright__ = "Me"
__credits__ = "Me"
__license__ = "Open"
__version__ = "0.0.7"
__maintainer__ = "Rahul Kakodkar"
__email__ = "Me"
__status__ = "Single"


**Import Modules**

In [1]:
from itertools import product
from pyomo.environ import *
import numpy 
import pandas 
from pyomo.opt import SolverStatus, TerminationCondition

**Declare Model Sets, Variables, Objective**

In [2]:
sudoku = ConcreteModel(doc = 'model for solving Sudoku') #declare the model instance

range_list = [i+1 for i in range(9)] #declare a handy list of numbers 1-9

sudoku.rows = Set(initialize = range_list, doc = 'set of rows')
sudoku.cols = Set(initialize = range_list, doc = 'set of columns') 
sudoku.vals = Set(initialize = range_list, doc = 'set of values that a box can take') 

sudoku.place = Var(sudoku.rows, sudoku.cols, sudoku.vals, within = Binary, doc = 'binary variable; 1 when a value is chosen in index')

**Declare Blocks**

In [3]:
block_dict = {f"{k}{l}": [(i,j) for j in range_list[3*k:3*k+3] for i in range_list[3*l:3*l+3]] for k,l in product(range(3), range(3))}
swap_keys = {list(block_dict.keys())[i]: i  for i in range(9)}
block_dict = {swap_keys[i] + 1: block_dict[i] for i in swap_keys.keys()}
block_dict #list of indices in each block

{1: [(1, 1), (2, 1), (3, 1), (1, 2), (2, 2), (3, 2), (1, 3), (2, 3), (3, 3)],
 2: [(4, 1), (5, 1), (6, 1), (4, 2), (5, 2), (6, 2), (4, 3), (5, 3), (6, 3)],
 3: [(7, 1), (8, 1), (9, 1), (7, 2), (8, 2), (9, 2), (7, 3), (8, 3), (9, 3)],
 4: [(1, 4), (2, 4), (3, 4), (1, 5), (2, 5), (3, 5), (1, 6), (2, 6), (3, 6)],
 5: [(4, 4), (5, 4), (6, 4), (4, 5), (5, 5), (6, 5), (4, 6), (5, 6), (6, 6)],
 6: [(7, 4), (8, 4), (9, 4), (7, 5), (8, 5), (9, 5), (7, 6), (8, 6), (9, 6)],
 7: [(1, 7), (2, 7), (3, 7), (1, 8), (2, 8), (3, 8), (1, 9), (2, 9), (3, 9)],
 8: [(4, 7), (5, 7), (6, 7), (4, 8), (5, 8), (6, 8), (4, 9), (5, 9), (6, 9)],
 9: [(7, 7), (8, 7), (9, 7), (7, 8), (8, 8), (9, 8), (7, 9), (8, 9), (9, 9)]}

**Fix values that are given**

In [4]:
sudoku_df = pandas.read_excel('sudoku.xlsx') #import fixed values from excel sheet
fix_dict = {(j+1,i+1): sudoku_df[i][j] for i,j in product(list(range(9)), list(range(9))) }

for i,j in product(range_list,range_list): #fix values when available 
    if fix_dict[(i,j)] > 0:
        sudoku.place[(i,j), fix_dict[(i,j)]].fixed = True
        sudoku.place[(i,j), fix_dict[(i,j)]].value = 1

sudoku_df 

,0,1,2,3,4,5,6,7,8
0,8,0,0,9,0,2,6,0,3
1,1,2,4,8,0,3,0,0,0
2,6,0,0,0,4,0,8,1,0
3,7,0,1,6,0,0,0,0,9
4,3,0,8,0,0,0,4,0,7
5,0,0,0,4,0,9,0,3,8
6,0,8,0,2,0,0,0,7,0
7,0,0,3,0,8,6,0,5,4
8,0,7,2,0,5,0,0,8,0


**Declare Constraints**

In [5]:

def place_rule(instance,rows, cols):
    return sum(instance.place[rows, cols, val_] for val_ in instance.vals) == 1

sudoku.place_rule = Constraint(sudoku.rows, sudoku.cols, rule = place_rule, doc = 'each place can have a number only once')

def row_sum_rule(instance, cols, vals):
    return sum(instance.place[row_, cols, vals] for row_ in instance.rows) == 1

sudoku.row_sum_cons = Constraint(sudoku.cols, sudoku.vals, rule = row_sum_rule, doc = 'each row can have a number only once')

def col_sum_rule(instance, rows, vals):
    return sum(instance.place[rows, col_, vals] for col_ in instance.cols) == 1

sudoku.col_sum_cons = Constraint(sudoku.cols, sudoku.vals, rule = col_sum_rule, doc = 'each col can have a number only once')

def block_cons_generator(instance, block): #Voodoo
    def block_rule(instance, vals):
        return sum(instance.place[block_, vals] for block_ in block) == 1
    return Constraint(instance.vals, rule = block_rule)

for i in range_list:   
    block_cons_generator(sudoku, block_dict[i])


**Solve Sudoku**

In [6]:
sudoku.fill = Objective(expr = 1)

result = SolverFactory('gurobi').solve(sudoku)

for i,j,k in product(range_list, range_list, range_list):
    if sudoku.place[i,j,k].value == 1:
        sudoku_df.loc[i-1,j-1] = k
sudoku_df.index = range_list
sudoku_df.columns = range_list

sudoku_df

    solver failure.


,1,2,3,4,5,6,7,8,9
1,8,5,7,9,1,2,6,4,3
2,1,2,4,8,6,3,7,9,5
3,6,3,9,5,4,7,8,1,2
4,7,4,1,6,3,8,5,2,9
5,3,9,8,1,2,5,4,6,7
6,2,6,5,4,7,9,1,3,8
7,5,8,6,2,9,4,3,7,1
8,9,1,3,7,8,6,2,5,4
9,4,7,2,3,5,1,9,8,6
